In [50]:
import math
import random
import numpy
from functools import reduce
import sys
import getopt
import re
import time

In [51]:
optimalFile = 'A-n32-k5.sol'
testFile = 'A-n32-k5.vrp'

In [52]:
def getDataFromFile(fileName):
    f = open(fileName, "r", encoding="utf-8")
    content = f.read()
    #print("Content: ", content)
    optimalValue = re.search("Optimal value: (\d+)", content, re.MULTILINE)
    if(optimalValue != None):
        optimalValue = optimalValue.group(1)
    else:
        optimalValue = re.search("Best value: (\d+)", content, re.MULTILINE)
        if(optimalValue != None):
            optimalValue = optimalValue.group(1)
    capacity = re.search("^CAPACITY : (\d+)$", content, re.MULTILINE).group(1)
    #graph = re.findall(r"^ (\d+) (\d+) (\d+)$", content, re.MULTILINE)
    graph = re.findall(r"^ (\d+) (\d+) (\d+)$", content, re.MULTILINE)
    dim = re.search("^DIMENSION : (\d+)$", content, re.MULTILINE).group(1)
    print("Graph: ", graph)
    dim = int(dim)
    print("dim: ", dim)
    #demand = re.findall(r"^(\d+) (\d+) $", content, re.MULTILINE)
    demand = re.findall(r"^(\d+) (\d+) $", content, re.MULTILINE)
    print("Demand: ", demand)
    graph = {int(a):(int(b),int(c)) for a,b,c in graph}
    print("graph: ", graph)
    demand = {int(a):int(b) for a,b in demand}
    print("demand: ", demand)
    capacity = int(capacity)
    optimalValue = int(optimalValue)
    return capacity, graph, demand, optimalValue, dim

In [53]:
alfa = 2 # importance of pheromones
beta = 7 # importance of distance
ro = 0.1 # speed of pheromon evaporation
th = 80 # importance of avg current solutions' costs
iterations = 1000
ants = 14

In [54]:
def generateGraph(capacityLimit, graph, demand, optimalValue):
    vertices = list(graph.keys())
    vertices.remove(1)

    edges = { (min(a,b),max(a,b)) : numpy.sqrt((graph[a][0]-graph[b][0])**2 + (graph[a][1]-graph[b][1])**2) for a in graph.keys() for b in graph.keys()}
    feromones = { (min(a,b),max(a,b)) : 1 for a in graph.keys() for b in graph.keys() if a!=b }
    
    return vertices, edges, capacityLimit, demand, feromones, optimalValue

In [55]:
import math

In [56]:
def solutionOfOneAnt(vertices, edges, capacityLimit, demand, feromones):
    solution = list()

    while(len(vertices)!=0):
        path = list()
        city = numpy.random.choice(vertices)
        capacity = capacityLimit - demand[city]
        path.append(city)
        vertices.remove(city)
        while(len(vertices)!=0):
            probabilities = list(map(lambda x: ((feromones[(min(x,city), max(x,city))])**alfa)*((1/edges[(min(x,city), max(x,city))])**beta), vertices))
            #print(probabilities)
            for i in range(len(probabilities)):
              if math.isinf(probabilities[i]):
                probabilities[i] = 1.2224513299473679e-11
            probabilities = probabilities/numpy.sum(probabilities)
            
            city = numpy.random.choice(vertices, p=probabilities)
            capacity = capacity - demand[city]

            if(capacity>0):
                path.append(city)
                vertices.remove(city)
            else:
                break
        solution.append(path)
    return solution

def rateSolution(solution, edges):
    s = 0
    for i in solution:
        a = 1
        for j in i:
            b = j
            s = s + edges[(min(a,b), max(a,b))]
            a = b
        b = 1
        s = s + edges[(min(a,b), max(a,b))]
    return s


In [57]:
def updateFeromone(feromones, solutions, bestSolution):
    # L - cost of the destance
    Lavg = reduce(lambda x,y: x+y, (i[1] for i in solutions))/len(solutions)
    #amount of feromones
    feromones = { k : (1 - ro) * v + th/Lavg for (k,v) in feromones.items() }
    solutions.sort(key = lambda x: x[1])
    if(bestSolution!=None):
        if(solutions[0][1] < bestSolution[1]):
            bestSolution = solutions[0]
    #    for path in bestSolution[0]:
     #       for i in range(len(path)-1):
      #          feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))] = sigm/bestSolution[1] + feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))]
    else:
        bestSolution = solutions[0]
    #for l in range(sigm):
     #   paths = solutions[l][0]
      #  L = solutions[l][1]
       # for path in paths:
        #    for i in range(len(path)-1):
         #       feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))] = (sigm-(l+1)/L**(l+1)) + feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))]
    return bestSolution

In [58]:
bestSolution = None
capacityLimit, graph, demand, optimalValue, dim = getDataFromFile(testFile)

Graph:  [('1', '82', '76'), ('2', '96', '44'), ('3', '50', '5'), ('4', '49', '8'), ('5', '13', '7'), ('6', '29', '89'), ('7', '58', '30'), ('8', '84', '39'), ('9', '14', '24'), ('10', '2', '39'), ('11', '3', '82'), ('12', '5', '10'), ('13', '98', '52'), ('14', '84', '25'), ('15', '61', '59'), ('16', '1', '65'), ('17', '88', '51'), ('18', '91', '2'), ('19', '19', '32'), ('20', '93', '3'), ('21', '50', '93'), ('22', '98', '14'), ('23', '5', '42'), ('24', '42', '9'), ('25', '61', '62'), ('26', '9', '97'), ('27', '80', '55'), ('28', '57', '69'), ('29', '23', '15'), ('30', '20', '70'), ('31', '85', '60'), ('32', '98', '5')]
dim:  32
Demand:  [('1', '0'), ('2', '19'), ('3', '21'), ('4', '6'), ('5', '19'), ('6', '7'), ('7', '12'), ('8', '16'), ('9', '6'), ('10', '16'), ('11', '8'), ('12', '14'), ('13', '21'), ('14', '16'), ('15', '3'), ('16', '22'), ('17', '18'), ('18', '19'), ('19', '1'), ('20', '24'), ('21', '8'), ('22', '12'), ('23', '4'), ('24', '8'), ('25', '24'), ('26', '24'), ('27', '2

In [59]:
vertices, edges, capacityLimit, demand, feromones, optimalValue = generateGraph(capacityLimit, graph, demand, optimalValue)

In [60]:
start_time = time.time()
for i in range(iterations):
  solutions = list()
  for _ in range(ants):
      solution = solutionOfOneAnt(vertices.copy(), edges, capacityLimit, demand, feromones)
      solutions.append((solution, rateSolution(solution, edges)))
  bestSolution = updateFeromone(feromones, solutions, bestSolution)
print("Solution: ", bestSolution) 
print("time of execution: %s seconds" %abs (time.time() - start_time)) # вычисление времени выполнения


Solution:  ([[29, 5, 12, 24, 4, 3, 7], [18, 20, 32, 22, 14, 8, 27], [30, 19, 9, 10, 23, 16, 11, 26, 6, 21], [31, 17, 2, 13, 15, 25], [28]], 823.5740219235631)
time of execution: 44.55662178993225 seconds


In [61]:
cost_from_alg = bestSolution[-1]
cost_from_alg

823.5740219235631

In [62]:
def getResFromFile(fileName):
  f = open(fileName, "r", encoding="utf-8")
  content = f.read()
  cost = re.search("^Cost (\d+)$", content, re.MULTILINE).group(1)
  return int(cost)

In [63]:
cost = getResFromFile(optimalFile)
cost

784

In [64]:
def compareResults(res1, res2):
  return abs(res1-res2)

In [65]:
compareResults(cost, cost_from_alg)

39.57402192356312

In [66]:
print("Time ", (abs (time.time() - start_time)))
print("Optimal ", cost)
print("In fact ", cost_from_alg)
print("Dim ", dim)

Time  44.61202812194824
Optimal  784
In fact  823.5740219235631
Dim  32
